## Functions


#### Libraries

In [1]:
import numpy as np
from numpy import *
from numpy.linalg import inv
from commpy.modulation import QAMModem
import keras.backend as K

Using TensorFlow backend.


#### Functions

In [2]:

def Basic_Nonlinear_Distortion_Model(A,B,q,p,G,Vsat,IBO):
    val_IBO_m1dB=((1/sqrt(10**-0.1))**(2*p)-1)**(1/(2*p))*Vsat/(G)
    coeff_IBO_m1dB=val_IBO_m1dB*sqrt(10**(-IBO/10))
    s=random.randn(1,1000000)
    vin1 = sqrt(1/2)*(s+1j*s)
    vin01 = coeff_IBO_m1dB*vin1
    a0=absolute(vin01)
    a02=a0**2  
    theta=angle(vin01)
    Am=(G*a0)/((1+(G*a0/Vsat)**(2*p))**(1/(2*p)))
    Bm=(A*(a0**q))/((1+(a0/B)**(q)))
    Sm=Am*exp(1j*(Bm))
    vout1=Am*exp(1j*(theta+Bm))
    K0 = mean(vout1*conj(vin01))/mean(absolute(vin01)**2)
    sigma2_d = var(vout1 - K0*vin01) 
    return(K0,sigma2_d)
Basic_Nonlinear_Distortion_Model.__doc__="""Cest une fonction qui répresente le modèle de distorsion non linéaire PA de base.
       Entré: Un train binaire aléatoire
       Sortie:
        -K0: Gain complexede PA
        -sigma2_d: Vecteur distortion de PA
"""

In [3]:
def hpa_sspa_modif_rapp(vin, Vsat, p, q, G, A, B):
    A = -345
    a0 = abs(vin)
    theta = np.angle(vin)
    Am = (G * a0) / ((1 + (G * a0 / Vsat) ** (2 * p)) ** (1 / (2 * p)))
    Bm = (A * (a0 ** q)) / ((1 + (a0 / B) ** (q)))
    vout = Am * np.exp(1j * (theta + Bm))
    return vout


hpa_sspa_modif_rapp.__doc__ = """ cette fonction permet de calculer la sortie de l'amplificateur de puissance PA.
        Le comportement non linéaire de PA peut être décrit en utilisant les caractéristiques de compression AM-AM et de conversion AM-PM
        Entré:
        -vin: Signal à amplifier 
        -Vsat: le niveau de saturation
        -p: le facteur de lissage
        -G: le petit gain de signal
        -q, A et B: paramètres d'ajustement
        Sortie: Signal amplifié
        """

In [4]:
def find_K0_sigma2_d(vin, vout):
    K0 = np.mean(vout * np.conj(vin)) / np.mean(np.absolute(vin) ** 2)
    sigma2_d = np.var(vout - K0 * vin)
    return (K0, sigma2_d)


find_K0_sigma2_d.__doc__ = """Cette fonction permet de trouver les parametres de distortion de non linearité de PA.
        D'apres la théorème de Bussgang, on peut décomposer le signal non linéaire à la sortie PA en une fonction linéaire de l'entrée PA et un terme de distorsion non corrélé.
        Le signal amplifié sera sous cette forme ymt = qmtxmt + dm
        Entré: 
        -vin: Signal à amplifier 
        -vout: Signal amplifié
        Sortie:
        -K0: Gain complexede PA
        -sigma2_d: Vecteur distortion de PA
        """

In [5]:
def log10(x):
    numerator = tf.math.log(x)
    denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator


log10.__doc__ = """Cette fonction permet de calculer le log10 d'un tenseur
       Entré: un tenseur
       Sortie: un tenseur
       """

In [6]:
def NMSE(X_train, y_pred):
    NMS = K.zeros(shape=(1000))
    NMSEdb = K.zeros(shape=(1000))
    # recieveh=K.zeros([1000,10])
    y_predt = tf.convert_to_tensor(y_pred, dtype=tf.float32)
    Y_gdr = y_predt[:1000, :100]
    Y_gdi = y_predt[:1000, 100:]
    Y_gd = tf.complex(Y_gdr, Y_gdi)
    X_testt = tf.convert_to_tensor(X_train, dtype=tf.float32)
    Shr = X_testt[:1000, :10]
    Shi = X_testt[:1000, 10:20]
    SSh = tf.complex(Shr, Shi)
    for i in range(1000):
        val_IBO_m1dB = (
            ((1 / np.sqrt(10 ** -0.1)) ** (2 * p) - 1) ** (1 / (2 * p)) * Vsat / (G)
        )
        coeff_IBO_m1dB = (
            val_IBO_m1dB
            * tf.math.sqrt((1 / K.var(Y_gd[i])))
            * np.sqrt(10 ** (-IBO / 10))
        )
        vin2 = coeff_IBO_m1dB * Y_gd[i]
        A = -345
        a0 = K.abs(vin2)
        theta = tf.math.angle(vin2)
        Am = (G * a0) / ((1 + (G * a0 / Vsat) ** (2 * p)) ** (1 / (2 * p)))
        Bm = (A * (a0 ** q)) / ((1 + (a0 / B) ** (q)))
        vout2 = tf.complex(Am, 0.0) * tf.math.exp(tf.complex(0.0, theta + Bm))
        Y_gd_amp = vout2 / coeff_IBO_m1dB
        Y_gd_amp0 = K.reshape(Y_gd_amp, (100, 1))
        HH = K.constant(H, dtype=tf.complex64)
        recieveh = K.dot(HH, (Y_gd_amp0))
        recievehh = K.reshape(recieveh, (1, 10))
        NMS = K.mean(K.abs(tf.math.subtract(recievehh, SSh[i]) ** 2)) / K.mean(
            K.abs(SSh[i]) ** 2
        )
        NMSEdb = 10 * log10(K.mean(NMS))
    return NMSEdb


NMSE.__doc__ = """Cette fonction permet de calculer l'erreur quadratique moyenne entre les symboles prévus S et les symboles amplifiés puis passés à travers le canal.
       Cette fonction répresente une metrique personnalisée compilée par la fonction compile() de la bibliothèque Keras.
       Entré:
       -Y_true: Un tenseur qui contient y_train
       -Y_Pred: Un tenseur qui contient y_pred
       Sortie: Un tenseur qui contient l'erreur quadratique moyenne en db.
       """

In [7]:
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))
root_mean_squared_error.__doc__="""
            Cette fonction permet de calculer la racine carrée de la moyenne des erreurs au carré.
            C'est la métrique la plus utilisée pour les tâches de régression.
            Il est préférable dans certains cas, car les erreurs sont d'abord quadrillées avant la moyenne, ce qui pose une pénalité élevée sur les grandes erreurs.
            Cela implique que RMSE est utile lorsque des erreurs importantes ne sont pas souhaitées.
            Entré:
            -Y_true: Un tenseur qui contient y_train
            -Y_Pred: Un tenseur qui contient y_pred
            Sortie: Un tenseur qui contient l'erreur quadratique moyenne.
             """